In [47]:
from langchain_aws import ChatBedrock
from langchain_aws import BedrockEmbeddings, AmazonKnowledgeBasesRetriever
import boto3

session = boto3.Session(profile_name="default")
bedrock_client = session.client('bedrock-runtime')

model_id = 'anthropic.claude-3-haiku-20240307-v1:0'
model_kwargs =  { 
        "max_tokens": 512,
        "temperature": 0,
        "top_p": 1,
}

# LangChain class for chat
model = ChatBedrock(
    client=bedrock_client,
    model_id=model_id,
    model_kwargs=model_kwargs,
)

embedding = BedrockEmbeddings(model_id="cohere.embed-english-v3",client=bedrock_client)


# Setup Models

# Add Product Pricing function tool

In [48]:
import pandas as pd
from langchain_core.tools import tool

#Load the laptop product pricing CSV into a Pandas dataframe.
product_pricing_df = pd.read_csv("data/Laptop pricing.csv")
print(product_pricing_df)

@tool
def get_laptop_price(laptop_name:str) -> int :
    """
    This function returns the price of a laptop, given its name as input.
    It performs a substring match between the input name and the laptop name.
    If a match is found, it returns the pricxe of the laptop.
    If there is NO match found, it returns -1
    """

    #Filter Dataframe for matching names
    match_records_df = product_pricing_df[
                        product_pricing_df["Name"].str.contains(
                                                "^" + laptop_name, case=False)
                        ]
    #Check if a record was found, if not return -1
    if len(match_records_df) == 0 :
        return -1
    else:
        return match_records_df["Price"].iloc[0]

#Test the tool. Before running the test, comment the @tool annotation
# print(get_laptop_price("alpha"))
print(get_laptop_price("GammaAir"))

            Name  Price  ShippingDays
0  AlphaBook Pro   1499             2
1     GammaAir X   1399             7
2  SpectraBook S   2499             7
3   OmegaPro G17   2199            14
4  NanoEdge Flex   1699             2
1399


# Add Product Features Retrieval Tool

In [49]:
# low level import using the name instead of (import pysqlite3)
__import__('pysqlite3')

import sys

# sys.modules is a dictionary that caches all imported modules
# sys.modules.pop('pysqlite3') removes the pysqlite3 module from the cache and returns it
# Every import sqlite3 in your code will actually load pysqlite3 instead
sys.modules['sqlite3'] = sys.modules.pop('pysqlite3') # 

# It appears it is asking chroma to use pysqllite3 instead of its default

from langchain.tools.retriever import create_retriever_tool
from langchain_chroma import Chroma 
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Load, chunk and index the contents of the product featuers document.
loader=PyPDFLoader("./data/Laptop product descriptions.pdf")
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=256)
splits = text_splitter.split_documents(docs)

#Create a vector store with Chroma
prod_feature_store = Chroma.from_documents(
    documents=splits, 
    embedding=embedding
)

get_product_features = create_retriever_tool(
    prod_feature_store.as_retriever(search_kwargs={"k": 1}),
    name="Get_Product_Features",
    description="""
    This store contains details about Laptops. It lists the available laptops
    and their features including CPU, memory, storage, design and advantages
    """
)

#Test the product feature store
# print(prod_feature_store.as_retriever().invoke("Tell me about the AlphaBook Pro") )

# Setup a Product QnA chatbot

In [50]:
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import AIMessage,HumanMessage,SystemMessage

#Create a System prompt to provide a persona to the chatbot
system_prompt = SystemMessage("""
    You are professional chatbot that answers questions about laptops sold by your company.
    To answer questions about laptops, you will ONLY use the available tools and NOT your own memory.
    You will handle small talk and greetings by producing professional responses.
    """
)

#Create a list of tools available
tools = [get_laptop_price, get_product_features]

#Create memory across questions in a conversation (conversation memory)
checkpointer=MemorySaver()

#Create a Product QnA Agent. This is actual a graph in langGraph
product_QnA_agent=create_react_agent(
                                model=model, #LLM to use
                                tools=tools, #List of tools to use
                                state_modifier=system_prompt, #The system prompt
                                debug=False, #Debugging turned on if needed
                                checkpointer=checkpointer #For conversation memory
)

In [42]:
#Setup chatbot
import uuid
#To maintain memory, each request should be in the context of a thread.
#Each user conversation will use a separate thread ID
config = {"configurable": {"thread_id": uuid.uuid4()}}

#Test the agent with an input
inputs = {"messages":[
                HumanMessage("What are the features for GammaAir?")
            ]}

#Use streaming to print responses as the agent  does the work.
#This is an alternate way to stream agent responses without waiting for the agent to finish
for stream in product_QnA_agent.stream(inputs, config, stream_mode="values"):
    message=stream["messages"][-1]
    if isinstance(message, tuple):
        print(message)
    else:
        message.pretty_print()

================================ Human Message =================================

What are the features for GammaAir?
================================== Ai Message ==================================

Okay, let me look up the features for the GammaAir laptop using the available tools.
Tool Calls:
  Get_Product_Features (toolu_bdrk_019uD79jnkxt1RpwdViqx6CF)
 Call ID: toolu_bdrk_019uD79jnkxt1RpwdViqx6CF
  Args:
    query: GammaAir
================================= Tool Message =================================
Name: Get_Product_Features

Fictional Laptop Descriptions
AlphaBook Pro
The AlphaBook Pro is a sleek ultrabook with a 12th Gen Intel i7 processor, 16GB of DDR4 RAM,
and a fast 1TB SSD. Ideal for professionals on the go, this laptop offers an impressive blend of
power and portability.
GammaAir X
GammaAir X combines an AMD Ryzen 7 processor with 32GB of DDR4 memory and a 512GB
NVMe SSD. Its thin and light form factor makes it perfect for users who need high performance in a
portable d

In [43]:
import uuid
#To maintain memory, each request should be in the context of a thread.
#Each user conversation will use a separate thread ID
config = {"configurable": {"thread_id": uuid.uuid4()}}

#Test the agent with an input
inputs = {"messages":[
                HumanMessage("What are the pricing for GammaAir?")
            ]}

#Use streaming to print responses as the agent  does the work.
#This is an alternate way to stream agent responses without waiting for the agent to finish
for stream in product_QnA_agent.stream(inputs, config, stream_mode="values"):
    message=stream["messages"][-1]
    if isinstance(message, tuple):
        print(message)
    else:
        message.pretty_print()

================================ Human Message =================================

What are the pricing for GammaAir?
================================== Ai Message ==================================
Tool Calls:
  get_laptop_price (toolu_bdrk_014BUTiwPrGEkxcqwJ4dcgPE)
 Call ID: toolu_bdrk_014BUTiwPrGEkxcqwJ4dcgPE
  Args:
    laptop_name: GammaAir
================================= Tool Message =================================
Name: get_laptop_price

1399
================================== Ai Message ==================================

The price of the GammaAir laptop is $1399.


# Execute the Product QnA Chatbot

In [51]:
import uuid
#Send a sequence of messages to chatbot and get its response
#This simulates the conversation between the user and the Agentic chatbot
user_inputs = [
    "Hello",
    "I am looking to buy a laptop",
    "Give me a list of available laptop names",
    "Tell me about the features of  SpectraBook",
    "How much does it cost?",
    "Give me similar information about OmegaPro",
    "What info do you have on AcmeRight ?",
    "Thanks for the help"
]

#Create a new thread
config = {"configurable": {"thread_id": str(uuid.uuid4())}}

for input in user_inputs:
    print(f"----------------------------------------\nUSER : {input}")
    #Format the user message
    user_message = {"messages":[HumanMessage(input)]}
    #Get response from the agent
    ai_response = product_QnA_agent.invoke(user_message,config=config)
    #Print the response
    print(f"AGENT : {ai_response['messages'][-1].content}")

----------------------------------------
USER : Hello
AGENT : Hello! How can I assist you today?
----------------------------------------
USER : I am looking to buy a laptop
AGENT : Great, I'd be happy to help you find the right laptop. What kind of features are you looking for in a laptop? For example, are there specific requirements around the processor, memory, storage, or design that are important to you?
----------------------------------------
USER : Give me a list of available laptop names
AGENT : Okay, based on the product information provided, it looks like the available laptop models are:

- AlphaBook Pro
- GammaAir X 
- SpectraBook S
- OmegaPro G17
- NanoEdge Flex

This covers a range of laptops with different processors, memory, storage, and design features to suit various user needs - from ultraportable productivity laptops to high-performance gaming and workstation machines. Let me know if you have any other specific requirements and I can try to recommend the best model 

In [44]:
#conversation memory by user
def execute_prompt(user, config, prompt):
    inputs = {"messages":[("user",prompt)]}
    ai_response = product_QnA_agent.invoke(inputs,config=config)
    print(f"\n{user}: {ai_response['messages'][-1].content}")

#Create different session threads for 2 users
config_1 = {"configurable": {"thread_id": str(uuid.uuid4())}}
config_2 = {"configurable": {"thread_id": str(uuid.uuid4())}}

#Test both threads
execute_prompt("USER 1", config_1, "Tell me about the features of  SpectraBook")
execute_prompt("USER 2", config_2, "Tell me about the features of  GammaAir")
execute_prompt("USER 1", config_1, "What is its price ?")
execute_prompt("USER 2", config_2, "What is its price ?")


USER 1: Based on the information provided, here are the key features of the SpectraBook S laptop:

- Processor: Intel Core i9 processor
- Memory: 64GB RAM
- Storage: 2TB SSD
- Designed for power users and intensive tasks like video editing and 3D rendering
- Workstation-class performance in a laptop form factor

The SpectraBook S seems to be a high-end, powerful laptop targeted at professionals who need a lot of processing power and storage for their demanding workloads. The large 64GB of RAM and 2TB SSD indicate this is a premium, top-of-the-line model in the SpectraBook lineup.

USER 2: Based on the information provided, the GammaAir X appears to have the following key features:

- AMD Ryzen 7 processor
- 32GB of DDR4 memory
- 512GB NVMe SSD storage
- Thin and light form factor, making it portable
- Designed for users who need high performance in a portable design

The description highlights that the GammaAir X combines powerful components like the AMD Ryzen 7 CPU and 32GB of RAM, w